## ライブラリ導入

In [ ]:
from qiskit import *
from qiskit.quantum_info import Operator, Statevector
from qiskit.visualization import *
from qiskit_aer import Aer
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy.optimize import minimize
import commonLogic

%matplotlib inline

## パラメータ設定

In [ ]:
# ノード数
num = 5
# エッジ構造
edges = [(0,1),(0,2),(1,2),(1,3),(2,4),(3,4)]
# 重み
weights = [1,1,1,1,1,1]
# トロッター分解数
p=2

# (γ,β)初期値
args_init = np.array([0.1,0.1,0.2,0.3])

In [ ]:
# コストハミルトニアン
def get_operator_costHamiltonian(num):
    Z = commonLogic.get_Z_list(num)
    dim = 2**num
    matrix = np.zeros((dim,dim))
    for k in range(len(edges)):
        i,j = edges[k]
        w = weights[k]
        matrix+=w*(np.ones((dim,dim)) - (Z[i]*Z[j]))
    matrix*=-1/2
    return Operator(matrix)

In [ ]:
# コストハミルトニアンの回路
def get_operator_cost(gamma):
    qc_cost = QuantumCircuit(num)

    for k in range(len(edges)):
        i,j = edges[k]
        w = weights[k]
        qc_cost.cx(i, j)
        qc_cost.rz(-2*gamma*w, j)
        qc_cost.cx(i, j)

    operator_cost = Operator(qc_cost)
    return operator_cost

In [ ]:
# ミキサーハミルトニアンの回路
def get_operator_mixer(beta):
    qc_mixer = QuantumCircuit(num)

    for i in range(num):
        qc_mixer.rx(-2*beta, i)

    operator_mixer = Operator(qc_mixer)
    return operator_mixer

## グラフ描画

In [ ]:
G = nx.Graph()
G.add_edges_from(edges)
pos = nx.spring_layout(G)
colors = ['aquamarine' for i in range(num)]
nx.draw(G, pos, with_labels=True, node_color=colors)
plt.show()

## 実装

In [ ]:
def get_mainCircuit(num,args):
    qc_main = QuantumCircuit(num,num)
    qc_main.h(range(num))
    for i in range(p):
        qc_main.append(get_operator_cost(args[i*2]),range(num))
        qc_main.append(get_operator_mixer(args[i*2+1]),range(num))
    qc_main.measure(range(num),range(num))
    return qc_main

def get_expectation_value(args):
    qc = get_mainCircuit(num,args)
    backend = Aer.get_backend("statevector_simulator")
    qc_transpiled = transpile(qc,backend)
    result = backend.run(qc).result()
    statevector = result.get_statevector()
    expectation_value = statevector.expectation_value(get_operator_costHamiltonian(num)).real
    return expectation_value

In [ ]:
optimizationResult = minimize(get_expectation_value, args_init, options={'maxiter':500}, method='powell')
print(optimizationResult.fun)
print(optimizationResult.x)

In [ ]:
gamma_optimized = optimizationResult.x[0]
beta_optimized = optimizationResult.x[1]

qc = get_mainCircuit(num,gamma_optimized,beta_optimized)
backend = Aer.get_backend("qasm_simulator")
qc_transpiled = transpile(qc,backend)
result = backend.run(qc).result()
counts=result.get_counts()
counts = dict(sorted(counts.items()))

commonLogic.my_plot_histogram(counts,gradation=True)

## 分割後のグラフ

In [ ]:
max_key = max(counts, key=counts.get)
colors = []
for i in reversed(max_key):
    if i=='0': colors.append('aquamarine')
    else: colors.append('red')
nx.draw(G, pos, with_labels=True, node_color=colors)
plt.show()